# Stats Processing

In [ ]:
import pandas as pd
import os, json


DATASET = "./GarminRawData"

In [ ]:
files = []
for dirName in os.listdir( DATASET ):

    basedir = os.path.join( DATASET, dirName )

    if os.path.isdir( basedir ):
        for fName in os.listdir( basedir ):
            if fName[-10:] == "-stat.json":
                fName = os.path.join( basedir, fName )
                print( f"""Found file: {fName}""" )
                files.append( fName )


In [ ]:
data = []

for fName in files:
    data.append( json.load( open( fName, 'rt') ) )


pd.DataFrame( data ).to_csv( "GarminStats.csv", index=False )